In [1]:
import numpy as np
f1 = open("D:/qiaohuijuan/test/lncRNA subcellular localization/code/featuresselectiondata/4kmerDSA1575KLNMF220.csv","rb")
features =np.loadtxt(f1,delimiter=',',skiprows=0)
f1.close()
features_selected= np.array(features)
#features_selected= np.column_stack((k_mer,DSA)) 
print(features_selected.shape)

(644, 220)


In [2]:
# Read labels
target= np.load("/target.npy")
print(target.shape)

(644,)


# PCA(copy=True, n_components=2, whiten=False)
from sklearn.decomposition import PCA
pca = PCA(n_components = 220)   #降到2维
pca.fit(features_new)                  #训练
features_selected=pca.fit_transform(features_selected)   #降维后的数据
#print(pca.explained_variance_ratio_)  #输出贡献率
print(features_selected.shape)          

# Apply the ANOVA to obtain the optimal feature subset
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
selector = SelectKBest(f_classif, k=220)
selector.fit(features_new, target)
features_selected = selector.transform(features_selected)
print(features_selected.shape)

In [3]:
#处理不平衡数据集
from imblearn.over_sampling import SMOTE
smo = SMOTE( random_state=42)
X_resampled_smote, y_resampled_smote = smo.fit_sample(features_selected, target)
features_selected = X_resampled_smote
target = y_resampled_smote
print(features_selected.shape)
print(target.shape)

(1668, 220)
(1668,)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import LeaveOneOut
loo = LeaveOneOut()
pipe = Pipeline([("scaler", MinMaxScaler()), 
                 ("svm",SVC(decision_function_shape="ovo",kernel="rbf",random_state=21))])

param_grid = {"svm__C":[pow(2, k) for k in range(-5, 15)],
             "svm__gamma":[pow(2, k) for k in range(-15, 5)]}

grid_search = GridSearchCV(pipe, param_grid, cv = 5,n_jobs=-1)
grid_search.fit(features_selected, target)
print(features_selected.shape)
# The best values for parameters C and gamma.
print(grid_search.best_params_)
print(grid_search.best_score_)